# Pytorch的单机多卡的训练

原始地址：https://pytorch.org/tutorials/beginner/blitz/data_parallel_tutorial.html

这个教程主要介绍的是如何利用多GPU卡来进行模型的训练，主要使用到的模块为`DataParallel`

在Pytorch中如果想把数据和模型拷贝到其他GPU上是非常容易的：

```python
device = torch.device("cuda:1")
model.to(device)
# 对于Tensor来说，返回的是在device上的新的tensor，而不是对原来tensor直接修改
mytensor = my_tensor.to(device)
```

如果我们希望在多块GPU上`forward/backward`我们的model,那我们就需要使用`DataParallel`了。

```python
model = nn.DataParallel(model)
```

下面开始我们的教程：

## 导入包和配置参数

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

input_size = 5
output_size = 2

batch_size = 30
data_size = 100

## 使用GPU设备

In [2]:
print('There are %d GPUs on the Node!' % (torch.cuda.device_count()))
device = torch.device('cuda:0')

There are 8 GPUs on the Node!


## 制作一个随机数填充的数据集

In [3]:
class RandomDataset(Dataset):
    def __init__(self, size, length):
        self.len = length
        # randn　标准正态分布的随机数引擎
        self.data = torch.randn(length, size)

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return self.len


rand_loader = DataLoader(dataset=RandomDataset(input_size, data_size),
                         batch_size=batch_size,
                         shuffle=True)

## 构造模型

这里为了测试，我们只创建了一个只有一层FC的网络，这不影响对`DateParallel`的使用。

In [4]:
class Model(nn.Module):
    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, input):
        output = self.fc(input)
        print("\tIn Model: input size", input.size(), "output size",
              output.size())
        return output

## 用`DataParallel`把模型分布式化

In [5]:
model = Model(input_size, output_size)
# device_ids参数可以指定使用哪些卡
# 注意device指定的gpu id 要与device_ids指定的卡的列表中的第一项匹配
model = nn.DataParallel(model, device_ids=[0,1])
device = torch.device('cuda:0')
model.to(device)

DataParallel(
  (module): Model(
    (fc): Linear(in_features=5, out_features=2, bias=True)
  )
)

In [6]:
for data in rand_loader:
    input = data.to(device)
    output = model(input)
    print("Outside: input size", input.size(), "output_size", output.size())

	In Model: input size	In Model: input size  torch.Size([15, 5])torch.Size([15, 5])  output sizeoutput size  torch.Size([15, 2])torch.Size([15, 2])

Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([15, 5]) 	In Model: input sizeoutput size  torch.Size([15, 5])Outside: input sizetorch.Size([15, 2])  
output sizetorch.Size([30, 5])  output_sizetorch.Size([15, 2]) 
	In Model: input size	In Model: input sizetorch.Size([30, 2])  
torch.Size([15, 5])torch.Size([15, 5])Outside: input size   output sizeoutput sizetorch.Size([30, 5])   torch.Size([15, 2])torch.Size([15, 2])	In Model: input size	In Model: input sizeoutput_size

   torch.Size([5, 5])torch.Size([5, 5])torch.Size([30, 2])  
output sizeoutput sizeOutside: input size   torch.Size([5, 2])torch.Size([5, 2])torch.Size([10, 5])

 output_size torch.Size([10, 2])


## 背后如何工作的

`DataParallel`根据卡数，把每一批(batch size)的数据平均等分，然后每一分片在一块GPU卡上运行Forward和Backward，然后再把结果合并在一起。内部使用的是单进程多线程的方式。

## 使用`DataParallel`后模型中的子模块

在单GPU下，我们可以通过下面的代码来单独使用Model的子模块

```python
model = Model(input_size, output_size)
out = model.fc(input)
```
但是在`DataParallel`中，需要修改为如下：
```python
model = nn.DataParallel(model)
out = model.module.fc(input)
```